In [2]:
#importing all the libraries that are needed. 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install -U pip
!pip install geopy
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import os
!pip install folium
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [31]:
# FourSquare API credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20210610' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('done')

done


In [ ]:
# Now going to get the LA Hotel Data. 

In [32]:
address = 'Los Angeles, US'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [45]:
search_query = 'hotels'
radius = 100000 # 10 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_hotels = requests.get(url).json()
hotels = result_hotels['response']['venues']
df_hotels = json_normalize(hotels)
print(df_hotels.shape)

(30, 18)


<ipython-input-45-ad8cc7ab4e37>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_hotels = json_normalize(hotels)


In [38]:
##Now going to clean the data. 

In [46]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_hotels.columns if col.startswith('location.')] + ['id']
df_hotels_filtered = df_hotels.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotels_filtered['categories'] = df_hotels_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotels_filtered.columns = [column.split('.')[-1] for column in df_hotels_filtered.columns]

pd.DataFrame(df_hotels_filtered)
df_hotels_filtered

,name,categories,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,address,crossStreet,id
0,Arrive Hotels & Resorts International Headquar...,General Travel,34.051426,-118.248137,"[{'label': 'display', 'lat': 34.05142637050482...",555,90013,US,Los Angeles,CA,United States,"[Los Angeles, CA 90013, United States]",NaN,NaN,502dffc2e4b01882f818fbb7
1,Hyatt Hotels and Resorts - Hyatt On Sunset,Food,34.047943,-118.258179,"[{'label': 'entrance', 'lat': 34.04754, 'lng':...",1558,90017,US,Los Angeles,CA,United States,"[711 S Hope St, Los Angeles, CA 90017, United ...",711 S Hope St,NaN,4f32258519836c91c7bababb
2,Mobilona Space Hotels,Office,34.050508,-118.259228,"[{'label': 'display', 'lat': 34.05050823419858...",1559,90017,US,Los Angeles,CA,United States,"[601 S Figueroa St Ste 4050, Los Angeles, CA 9...",601 S Figueroa St Ste 4050,NaN,50e00b3f011ce3f4d5913aee
3,D Pet Hotels,Hotel,34.090130,-118.338661,"[{'label': 'display', 'lat': 34.09012989011778...",9728,90038,US,Los Angeles,CA,United States,"[1041 N Highland Ave (Santa Monica Blvd), Los ...",1041 N Highland Ave,Santa Monica Blvd,4b55089ff964a520a2d827e3
4,Omni Los Angeles Hotel at California Plaza,Hotel,34.052497,-118.250455,"[{'label': 'display', 'lat': 34.05249694980792...",721,90012,US,Los Angeles,CA,United States,"[251 S Olive St, Los Angeles, CA 90012, United...",251 S Olive St,NaN,4a6be87ef964a5203ad01fe3
5,D Pet Hotels Los Angeles,Pet Service,34.162424,-118.516058,"[{'label': 'display', 'lat': 34.16242377115171...",27946,91316,US,Encino,CA,United States,"[17553 Ventura Blvd, Encino, CA 91316, United ...",17553 Ventura Blvd,NaN,57e4d2b7cd1095109c9a843d
6,Wag Hotels,Pet Service,34.033319,-118.446662,"[{'label': 'display', 'lat': 34.033319, 'lng':...",18943,90064,US,Los Angeles,CA,United States,"[2250 S Barrington Ave, Los Angeles, CA 90064,...",2250 S Barrington Ave,NaN,5d4cc8fb1fd1990008343e32
7,"Hotel Solaire, Los Angeles",Hotel,34.054414,-118.272810,"[{'label': 'display', 'lat': 34.05441352829405...",2772,90017,US,Los Angeles,CA,United States,"[1710 W 7th St, Los Angeles, CA 90017, United ...",1710 W 7th St,NaN,507b3278498e98df0fcccd31
8,hotels,Hostel,33.896232,-118.308731,"[{'label': 'display', 'lat': 33.89623186524289...",18555,90249,US,Gardena,CA,United States,"[Gardena, CA 90249, United States]",NaN,NaN,4f125708e4b09e58a20152a5
9,W Hotels The Store,Boutique,34.101202,-118.325626,"[{'label': 'display', 'lat': 34.10120194621137...",9292,90028,US,Los Angeles,CA,United States,"[6250 Hollywood Blvd, Hollywood, CA 90028, Uni...",6250 Hollywood Blvd,NaN,4d6d836ecb7a8cfac7a36079


In [68]:
# I noticed that a food, office, Pool and None are hotels too. Lets fix that

df_hotels_filtered = df_hotels_filtered[df_hotels_filtered.categories.isin(['Hotel', 'Office', 'Food', 'None', 'Pool'])]
df_hotels_filtered.categories = 'Hotel'
df_hotels_filtered.reset_index(drop=True, inplace=True)
df_hotels_filtered

AttributeError: 'DataFrame' object has no attribute 'categories'

In [69]:
print(df_hotels_filtered.shape)

(16, 7)


In [66]:
df_hotels_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'cc', 'categories', 'country', 'city'], axis=1, inplace=True)
print(df_hotels_filtered)

                                                 name        lat         lng  \
0          Hyatt Hotels and Resorts - Hyatt On Sunset  34.047943 -118.258179   
1                               Mobilona Space Hotels  34.050508 -118.259228   
2                                        D Pet Hotels  34.090130 -118.338661   
3          Omni Los Angeles Hotel at California Plaza  34.052497 -118.250455   
4                          Hotel Solaire, Los Angeles  34.054414 -118.272810   
5                       Hotel Silver Lake Los Angeles  34.075928 -118.285149   
6                                          Hotel Samy  34.091939 -118.345918   
7           Andaz West Hollywood - a concept by Hyatt  34.095285 -118.373245   
8                                      Hotel Sweetzer  34.074179 -118.370088   
9                              Californian Hotel Sign  34.114890 -118.270652   
10                                           K Hotels  34.146219 -118.417447   
11                        The Sophia Hot

In [67]:
allhotels_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Los Angeles

# add a red circle marker to represent the Los Angeles
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Los Angeles',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(allhotels_map)

# add the Hotels as blue circle markers
for lat, lng, label in zip(df_hotels_filtered.lat, df_hotels_filtered.lng, df_hotels_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(allhotels_map)

# display map
allhotels_map

In [70]:
#there are still a few hotels to pick from, lets narrow it down with rating 
ratings = pd.DataFrame()
list1 = pd.Series([]) 
list2 = pd.Series([]) 
for i in range(len(df_hotels_filtered.id)):
    hotel_id = df_hotels_filtered.id[i] # ID of Hotels
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(hotel_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #Let's Check for Ratings, If Available then store it and if not, then store 0
    try:
        list1[i] = result['response']['venue']['rating'] 
        list2[i] = df_hotels_filtered.id[i]
    except:
        list1[i] = 0
        list2[i] = df_hotels_filtered.id[i]
         
        
ratings.insert(0, "ID", list2, allow_duplicates=True)
ratings.insert(1, "Ratings", list1, allow_duplicates=True)
ratings

<ipython-input-70-0d4ff03cb363>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  list1 = pd.Series([])
<ipython-input-70-0d4ff03cb363>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  list2 = pd.Series([])


,ID,Ratings
0,4f32258519836c91c7bababb,0.0
1,50e00b3f011ce3f4d5913aee,0.0
2,4b55089ff964a520a2d827e3,0.0
3,4a6be87ef964a5203ad01fe3,6.5
4,507b3278498e98df0fcccd31,5.7
5,59a9a6fdb9ac386d8dd96c7e,0.0
6,4b5fc9d0f964a520bccc29e3,0.0
7,49ebd48df964a52037671fe3,8.2
8,4cb3543bef1b3704f3ab5100,0.0
9,514e08abe4b08debe58855ee,0.0


In [71]:
#merge both dataframes based on id
ratings.columns = ['id', 'Ratings']
df_hotels_filtered = pd.merge(df_hotels_filtered,
                 ratings,
                 on='id')
df_hotels_filtered

,name,lat,lng,distance,formattedAddress,address,id,Ratings
0,Hyatt Hotels and Resorts - Hyatt On Sunset,34.047943,-118.258179,1558,"[711 S Hope St, Los Angeles, CA 90017, United ...",711 S Hope St,4f32258519836c91c7bababb,0.0
1,Mobilona Space Hotels,34.050508,-118.259228,1559,"[601 S Figueroa St Ste 4050, Los Angeles, CA 9...",601 S Figueroa St Ste 4050,50e00b3f011ce3f4d5913aee,0.0
2,D Pet Hotels,34.090130,-118.338661,9728,"[1041 N Highland Ave (Santa Monica Blvd), Los ...",1041 N Highland Ave,4b55089ff964a520a2d827e3,0.0
3,Omni Los Angeles Hotel at California Plaza,34.052497,-118.250455,721,"[251 S Olive St, Los Angeles, CA 90012, United...",251 S Olive St,4a6be87ef964a5203ad01fe3,6.5
4,"Hotel Solaire, Los Angeles",34.054414,-118.272810,2772,"[1710 W 7th St, Los Angeles, CA 90017, United ...",1710 W 7th St,507b3278498e98df0fcccd31,5.7
5,Hotel Silver Lake Los Angeles,34.075928,-118.285149,4626,"[250 Silver Lake Blvd (W Temple St), Los Angel...",250 Silver Lake Blvd,59a9a6fdb9ac386d8dd96c7e,0.0
6,Hotel Samy,34.091939,-118.345918,10420,"[1140 N Formosa Ave (at Santa Monica Blvd), We...",1140 N Formosa Ave,4b5fc9d0f964a520bccc29e3,0.0
7,Andaz West Hollywood - a concept by Hyatt,34.095285,-118.373245,12891,"[8401 Sunset Blvd, West Hollywood, CA 90069, U...",8401 Sunset Blvd,49ebd48df964a52037671fe3,8.2
8,Hotel Sweetzer,34.074179,-118.370088,11960,"[Los Angeles, CA 90048, United States]",NaN,4cb3543bef1b3704f3ab5100,0.0
9,Californian Hotel Sign,34.114890,-118.270652,7281,"[Los Angeles, CA, United States]",NaN,514e08abe4b08debe58855ee,0.0


In [73]:
#Sorting the Ratings
df_hotels_filtered.sort_values('Ratings', ascending=False, inplace=True) 

#Make new dataframe for top hotels
df_top_hotels = df_hotels_filtered.head(10)
df_top_hotels

,name,lat,lng,distance,formattedAddress,address,id,Ratings
7,Andaz West Hollywood - a concept by Hyatt,34.095285,-118.373245,12891,"[8401 Sunset Blvd, West Hollywood, CA 90069, U...",8401 Sunset Blvd,49ebd48df964a52037671fe3,8.2
14,W Los Angeles - West Beverly Hills,34.063275,-118.440950,18308,"[930 Hilgard Ave (at Le Conte Ave), Los Angele...",930 Hilgard Ave,43d39576f964a520202e1fe3,7.3
3,Omni Los Angeles Hotel at California Plaza,34.052497,-118.250455,721,"[251 S Olive St, Los Angeles, CA 90012, United...",251 S Olive St,4a6be87ef964a5203ad01fe3,6.5
4,"Hotel Solaire, Los Angeles",34.054414,-118.272810,2772,"[1710 W 7th St, Los Angeles, CA 90017, United ...",1710 W 7th St,507b3278498e98df0fcccd31,5.7
15,Sheraton Pasadena Hotel,34.143460,-118.144221,13504,"[303 Cordova St, Pasadena, CA 91101, United St...",303 Cordova St,4aea6fd6f964a52093bb21e3,5.4
0,Hyatt Hotels and Resorts - Hyatt On Sunset,34.047943,-118.258179,1558,"[711 S Hope St, Los Angeles, CA 90017, United ...",711 S Hope St,4f32258519836c91c7bababb,0.0
1,Mobilona Space Hotels,34.050508,-118.259228,1559,"[601 S Figueroa St Ste 4050, Los Angeles, CA 9...",601 S Figueroa St Ste 4050,50e00b3f011ce3f4d5913aee,0.0
2,D Pet Hotels,34.090130,-118.338661,9728,"[1041 N Highland Ave (Santa Monica Blvd), Los ...",1041 N Highland Ave,4b55089ff964a520a2d827e3,0.0
5,Hotel Silver Lake Los Angeles,34.075928,-118.285149,4626,"[250 Silver Lake Blvd (W Temple St), Los Angel...",250 Silver Lake Blvd,59a9a6fdb9ac386d8dd96c7e,0.0
6,Hotel Samy,34.091939,-118.345918,10420,"[1140 N Formosa Ave (at Santa Monica Blvd), We...",1140 N Formosa Ave,4b5fc9d0f964a520bccc29e3,0.0


In [ ]:
#Since my work is going be in downtown LA, I'm going to go with Omni Los Angeles Hotel, to avoid the traffic. The distance is the lowest and its number 3 in Ratings. 